In [1]:
import cifar10_input
import numpy as np
import tensorflow as tf
import time
import os

In [2]:
cifar10_input.maybe_download_and_extract()

>> Downloading cifar-10-binary.tar.gz 100.0%
Successfully downloaded cifar-10-binary.tar.gz 170052171 bytes.


In [3]:
# Hyperparams
# Architecture
N_HIDDEN_1 = 256
N_HIDDEN_2 = 256
ETA = 0.001
EPOCHS = 1000
BATCH_SIZE = 128
DISPLAY_STEP = 1

In [4]:
def inputs(eval_data=True):
    data_dir = os.path.join('data/cifar10_data', 'cifar-10-batches-bin')
    return cifar10_input.inputs(
        eval_data=eval_data, data_dir=data_dir, batch_size=BATCH_SIZE)

In [6]:
def distorted_inputs():
    data_dir = os.path.join('data/cifar10_data', 'cifar-10-batches-bin')
    return cifar10_input.distorted_inputs(data_dir=data_dir, 
                                          batch_size=BATCH_SIZE)

In [7]:
def filter_summary(V, weight_shape):
    ix = weight_shape[0] # ??? not used
    iy = weight_shape[1] # ???
    cx, cy = 8, 8        # ???
    V_T = tf.transpose(V, (3, 0, 1, 2)) # magic numbers!
    tf.image_summary('filters', V_T, max_images=64)

In [8]:
def conv2d(input, weight_shape, bias_shape, visualize=False):
    incoming = weight_shape[0] * weight_shape[1] * weight_shape[2]
    weight_init = tf.random_normal_initializer(
        stddev=(2. / incoming) ** 0.5)
    bias_init = tf.constant_initializer(value=0)
    W = tf.get_variable('W', weight_shape, initializer=weight_init)
    b = tf.get_variable('b', bias_shape, initializer=bias_init)
    if visualize: filter_summary(W, weight_shape)
    return tf.nn.relu(tf.nn.bias_add(
        tf.nn.conv2d(
            input, W, strides=[1, 1, 1, 1], padding='SAME'), 
        b))

In [9]:
def max_pool(input, k=2):
    return tf.nn.max_pool(
        input, ksize=[1, k, k, 1], strides=[1, k, k, 1], padding='SAME')

In [10]:
def layer(input, weight_shape, bias_shape):
    weight_init = tf.random_normal_initializer(
        stddev=(2. / weight_shape[0]) ** 0.5)
    bias_init = tf.constant_initializer(value=0)
    W = tf.get_variable('W', weight_shape, initializer=weight_init)
    b = tf.get_variable('b', bias_shape, initializer=bias_init)
    return tf.nn.relu(tf.matmul(input, W) + b)

In [ ]:
def inference(x, keep_prob):
    